In [ ]:
import pandas as pd
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [ ]:
def read_input(file_name):
    with open(file_name, 'r') as file:
        n, m = map(int, file.readline().strip().split())
        data = [tuple(map(int, line.strip().split() + [i])) for i,line in enumerate(file.readlines())]
    return n, m, data[:n], data[n:]

In [ ]:
def sort_missions(missions):
    missions_sorted = np.array(sorted(missions, key=lambda x: x[1]))
    start_date = np.array(sorted(missions, key=lambda x: x[0]))[0,0]
    return missions_sorted, start_date


In [ ]:
def sort_chronobots_init(chronobots, start_date, n, tuning):
    chronobots_sorted = pd.DataFrame(data = sorted(chronobots, key=lambda x: x[2], reverse=True))

    chronobots_sorted_top = chronobots_sorted.loc[chronobots_sorted[2] > 100]
    chronobots_sorted_top[4] = tuning
    chronobots_sorted_top[4] = chronobots_sorted_top[[2, 4]].min(axis=1)
    chronobots_sorted_top.sort_values([4, 0], ascending = [False, True], inplace=True)


    chronobots_sorted_bottom = chronobots_sorted.loc[chronobots_sorted[2] <= 100]
    chronobots_sorted_bottom.sort_values([0, 2], ascending = [False, False], inplace=True)
    chronobots_sorted = np.array(pd.concat([chronobots_sorted_top, chronobots_sorted_bottom], sort=False).loc[:,0:3])
    chronobots_sorted[:,3] -= n


    chronobots_sorted = np.c_[ chronobots_sorted, np.ones(len(chronobots_sorted), dtype = np.int64)*start_date ]
    chronobots_sorted = np.c_[ chronobots_sorted, np.zeros(len(chronobots_sorted), dtype = np.int64) ]

    chronobots_sorted = chronobots_sorted.astype('int64')

    k = len(chronobots_sorted_top)

    return chronobots_sorted, k


In [ ]:
def sort_chronobots(chronobots):
    chronobots_sorted = pd.DataFrame(data = chronobots)

    chronobots_sorted.sort_values([5], ascending = False, inplace=True)

    chronobots_sorted = np.array(chronobots_sorted)
    chronobots_sorted = chronobots_sorted.astype('int64')

    return chronobots_sorted

In [ ]:
def calculate_mission_revenue(r, c, k, is_chronobot_overworked):
    if is_chronobot_overworked == False:
        return r
    else:
        return min(int(r * (c*0.01) ** (k+1)), 2 * r)

In [ ]:
def get_unique_dates(missions_sorted):
    unique_dates = sorted(set(np.append(missions_sorted[:, 0], missions_sorted[:, 1])))
    return(unique_dates)

In [ ]:
def save_solution(results):
    total_revenue = 0
    sol = [0]*n
    for idx in results.keys():
        total_revenue += results[idx][0]
        for a in results[idx][1]:
            sol[a] = idx+1

    with open("output.txt", 'w') as file:
        file.write(' '.join(map(str, sol)))
    print("total_revenue = ", total_revenue)

In [ ]:
def calculate_chronobot_revenue(chronobot, missions_sorted, unique_dates):
    # Initialize dynamic programming table
    dp = {chronobot[4]: [0, 0, 0, []]}
    mission_index = 0

    # Iterate through unique_dates
    for i, date in enumerate(unique_dates):
        if date < chronobot[4]:
            pass
        dp_entry = dp[date]

        # Iterate through missions with the current end date
        # mission_index < len(missions_sorted) and
        while mission_index < len(missions_sorted) and missions_sorted[mission_index][1] == date:
            mission = missions_sorted[mission_index]
            prev_dp_entry = dp[mission[0]]
            is_chronobot_overworked = prev_dp_entry[2] + mission[1] - mission[0] > chronobot[0]

            # Calculate mission revenue
            mission_revenue = calculate_mission_revenue(mission[2], chronobot[2], prev_dp_entry[1], is_chronobot_overworked)

            # Update the dp_entry if there is a better revenue
            if (mission_revenue + prev_dp_entry[0]) > dp_entry[0]:
                dp_entry = [
                    mission_revenue + prev_dp_entry[0],
                    prev_dp_entry[1] + is_chronobot_overworked,
                    prev_dp_entry[2] + mission[1] - mission[0],
                    prev_dp_entry[3] + [mission[3]]
                ]
            mission_index += 1

        # Update dp table
        dp[unique_dates[min(i + 1, len(unique_dates)-1)]] = dp_entry

    # Calculate final revenue and mission indices for the given chronobot

    final_dp_entry = dp[unique_dates[-1]]
    final_revenue = final_dp_entry[0] - chronobot[1] * (final_dp_entry[1] != 0)
    mission_indices = final_dp_entry[3]

    return final_revenue, mission_indices


In [ ]:
def calculate_chronobot_revenue_no_overwork(chronobot, missions_sorted, unique_dates):
    # Initialize dynamic programming table
    dp = {chronobot[4]: [0, 0, 0, []]}
    mission_index = 0
    for date in unique_dates:
        dp[date] = [0, 0, 0, []]

    # Iterate through unique_dates
    for i, date in enumerate(unique_dates):
        if date < chronobot[4]:
            pass
        dp_entry = dp[date]

        # Iterate through missions with the current end date
        # mission_index < len(missions_sorted) and
        while mission_index < len(missions_sorted) and missions_sorted[mission_index][1] == date:
            mission = missions_sorted[mission_index]
            prev_dp_entry = dp[mission[0]]
            is_chronobot_overworked = prev_dp_entry[2] + mission[1] - mission[0] > chronobot[0]

            # Calculate mission revenue
            mission_revenue = calculate_mission_revenue(mission[2], chronobot[2], prev_dp_entry[1], is_chronobot_overworked)

            # Skip mission if it would cause the chronobot to become overworked
            if is_chronobot_overworked:
                mission_index += 1
                continue


            # Update the dp_entry if there is a better revenue
            if (mission_revenue + prev_dp_entry[0]) > dp_entry[0]:
                dp_entry = [
                    mission_revenue + prev_dp_entry[0],
                    prev_dp_entry[1] + is_chronobot_overworked,
                    prev_dp_entry[2] + mission[1] - mission[0],
                    prev_dp_entry[3] + [mission[3]]
                ]
            mission_index += 1
        # Update dp table
        dp[unique_dates[min(i + 1, len(unique_dates)-1)]] = dp_entry

    # Calculate final revenue and mission indices for the given chronobot
    # if dp == {0: [0, 0, 0, []]}:
    #     return 0, []
    final_dp_entry = dp[unique_dates[-1]]
    final_revenue = final_dp_entry[0] - chronobot[1] * (final_dp_entry[1] != 0)
    mission_indices = final_dp_entry[3]

    return final_revenue, mission_indices

In [ ]:
def find_best_chronobot(missions_sorted, chronobots_sorted, unique_dates):
    best_chronobot = None
    best_revenue = float('-inf')
    best_mission_indices = []

    for chronobot in chronobots_sorted:
        # Calculate the revenue and mission indices for the current chronobot

        revenue, mission_indices = calculate_chronobot_revenue(chronobot, missions_sorted, unique_dates)

        # Update the best chronobot if the current chronobot has a higher revenue
        if revenue > best_revenue:
            best_chronobot = chronobot
            best_revenue = revenue
            best_mission_indices = mission_indices

    return best_chronobot, best_revenue, best_mission_indices

In [ ]:
def find_best_chronobot_order(missions_sorted, chronobots_sorted, unique_dates):

    for chronobot in chronobots_sorted:

        if chronobot[2] > 89:
            best_revenue, mission_indices = calculate_chronobot_revenue(chronobot, missions_sorted, unique_dates)
            best_revenue_n_o, mission_indices_n_o = calculate_chronobot_revenue_no_overwork(chronobot, missions_sorted, unique_dates)

            if best_revenue <= best_revenue_n_o:
                best_revenue, mission_indices = best_revenue_n_o, mission_indices_n_o

        else :
            best_revenue, mission_indices = calculate_chronobot_revenue_no_overwork(chronobot, missions_sorted, unique_dates)

        chronobot[5] = best_revenue

    chronobots_sorted = sort_chronobots(chronobots_sorted)

    return chronobots_sorted

In [ ]:
def optimize_revenue(n, m, missions_sorted, chronobots_sorted, results, unique_dates, missions_done, take_first):
    # If all chronobots are processed, return the results

    if m == 0:
        return results, missions_sorted
    # Find the best chronobot and its revenue and mission indices
    if take_first:
        best_chronobot = chronobots_sorted[0]
        best_revenue, best_mission_indices = calculate_chronobot_revenue(best_chronobot, missions_sorted, unique_dates)
    else:
        best_chronobot, best_revenue, best_mission_indices = find_best_chronobot(missions_sorted, chronobots_sorted, unique_dates)
    # Update the results for the best chronobot

    temp = {best_chronobot[3]:[best_revenue, best_mission_indices]}
    with open('cr/'+str(best_chronobot[3])+'cr3.pickle', 'wb') as handle:
        pickle.dump(temp, handle, protocol=pickle.HIGHEST_PROTOCOL)
    results[best_chronobot[3]] = [best_revenue, best_mission_indices]

    # Remove the best chronobot from the sorted list
    for idx, chronobot in enumerate(chronobots_sorted):
        if chronobot[3] == best_chronobot[3]:
            chronobots_sorted = np.delete(chronobots_sorted, idx, 0)
            break

    # Update mission completions and remove completed missions
    completed_mission_indices = []

    for mission_completed in results[best_chronobot[3]][1]:
        missions_done[mission_completed] += 1
    for idx, mission in enumerate(missions_sorted):
        if missions_done[mission[3]] == 1:
            completed_mission_indices.append(idx)
    missions_sorted = np.delete(missions_sorted, completed_mission_indices, 0)

    # Update_unique dates
    unique_dates = get_unique_dates(missions_sorted)

    # Recursively process the remaining chronobots
    return optimize_revenue(n, m - 1, missions_sorted, chronobots_sorted, results, unique_dates, missions_done, take_first)

In [ ]:
file_name = "input_4c.txt"
tuning = 130
n, m, missions, chronobots = read_input(file_name)


missions_sorted, start_date = sort_missions(missions)
chronobots_sorted, k = sort_chronobots_init(chronobots, start_date, n, tuning)
unique_dates = get_unique_dates(missions_sorted)
results = {}
sol = {}
missions_done = [0]*(n+1)
chronobot = chronobots_sorted[0]
# Find the best chronobot order
chronobots_sorted = find_best_chronobot_order(missions_sorted, chronobots_sorted, unique_dates)


save = -1
while len(chronobots_sorted) != 0:
    save += 1
    top_chronobots = find_best_chronobot_order(missions_sorted, chronobots_sorted[:12], unique_dates)

    best_chronobot = top_chronobots[0]
    results = {}


    if best_chronobot[2] > 89:
        best_revenue, mission_indices = calculate_chronobot_revenue(best_chronobot, missions_sorted, unique_dates)
        best_revenue_n_o, mission_indices_n_o = calculate_chronobot_revenue_no_overwork(best_chronobot, missions_sorted, unique_dates)

        if best_revenue <= best_revenue_n_o:

            best_revenue, mission_indices = best_revenue_n_o, mission_indices_n_o
        best_chronobot[5] = best_revenue
    else :
        best_revenue, mission_indices = calculate_chronobot_revenue_no_overwork(best_chronobot, missions_sorted, unique_dates)
        best_chronobot[5] = best_revenue

    results[best_chronobot[3]] = [best_revenue, mission_indices]
    # Update mission completions and remove completed missions
    missions_done = [0]*(n+1)
    completed_mission_indices = []


    for mission_completed in mission_indices:
        missions_done[mission_completed] += 1
    for idx, mission in enumerate(missions_sorted):
        if missions_done[mission[3]] == 1:
            completed_mission_indices.append(idx)
    missions_sorted = np.delete(missions_sorted, completed_mission_indices, 0)


    with open('crs/'+str(best_chronobot[3])+'cr3.pickle', 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    for key in results.keys():
        sol[key] = results[key]

    chronobots_sorted = np.concatenate(([top_chronobots[1:], chronobots_sorted[12:]]), axis=0)
save_solution(sol)
with open('crs/nmcr3.pickle', 'wb') as handle:
    pickle.dump((n,m), handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
file_name = "input_4c.txt"
tuning = 125
tuning2 = 30
n, m, missions, chronobots = read_input(file_name)


# with open('temp.pickle', 'rb') as handle:
#     results_old = pickle.load(handle)
# #
# used_cr = []
# used_missions = []
#
# for cr in resul ts_old.keys():
#     used_cr += [cr]
#     used_missions += results_old[cr][1]
#
# chronobots = np.delete(chronobots, used_cr, 0)
# missions = np.delete(missions, used_missions, 0)


missions_sorted, start_date = sort_missions(missions)
chronobots_sorted, k = sort_chronobots_init(chronobots, start_date, n, tuning)
unique_dates = get_unique_dates(missions_sorted)
results = {}
sol = {}


# chronobot = chronobots_sorted[0]
# r1, temp = calculate_chronobot_revenue(chronobot, missions_sorted, unique_dates)

# best_chronobot, best_revenue, best_mission_indices = find_best_chronobot(missions_sorted, chronobots_sorted, unique_dates)
results = {}
count = 0
for chronobot in chronobots_sorted:
    count += 1

    if chronobot[2] > tuning2:
        best_revenue, mission_indices = calculate_chronobot_revenue(chronobot, missions_sorted, unique_dates)
        best_revenue_n_o, mission_indices_n_o = calculate_chronobot_revenue_no_overwork(chronobot, missions_sorted, unique_dates)

        if best_revenue <= best_revenue_n_o:

            best_revenue, mission_indices = best_revenue_n_o, mission_indices_n_o

    else :
        best_revenue, mission_indices = calculate_chronobot_revenue_no_overwork(chronobot, missions_sorted, unique_dates)


    results[chronobot[3]] = [best_revenue, mission_indices]

    # Update mission completions and remove completed missions
    missions_done = [0]*(n+1)
    completed_mission_indices = []


    for mission_completed in mission_indices:
        missions_done[mission_completed] += 1
    for idx, mission in enumerate(missions_sorted):
        if missions_done[mission[3]] == 1:
            completed_mission_indices.append(idx)
    missions_sorted = np.delete(missions_sorted, completed_mission_indices, 0)


    # with open('crs/'+str(best_chronobot[3])+'cr3.pickle', 'wb') as handle:
    #     pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # for key in results.keys():
    #     sol[key] = results[key]


s = 0
final2 = [0]*n
for idx in results.keys():

    s += results[idx][0]
    for a in results[idx][1]:

        final2[a-1] = idx

with open("output.txt", 'w') as file:
    file.write(' '.join(map(str, final2)))
print("s = ", s)

In [ ]:
s = 0
final2 = [0]*n
for idx in results.keys():

    s += results[idx][0]
    for a in results[idx][1]:

        final2[a] = idx+1

with open("output.txt", 'w') as file:
    file.write(' '.join(map(str, final2)))
print("s = ", s)